In [1]:
import os

import pandas as pd
import requests
from typing import List
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential
from concurrent.futures import ThreadPoolExecutor, as_completed

import os
import matplotlib.pyplot as plt
from skimage import io
import seaborn as sns
import warnings
import numpy as np
import warnings
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pylab import mpl, plt
import matplotlib.patches as mpatches

# best font and style settings for notebook 
warnings.filterwarnings('ignore')
sns.set_style("white")
mpl.rcParams['font.family'] = 'MiSans'


@retry(stop=stop_after_attempt(7), wait=wait_exponential(multiplier=2, min=2, max=100))
def post_with_retry(payload, headers, API_URL):
    response = requests.post(API_URL, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()


def analyze_abnormal_logs(df_abnormal, api_keys: List[str], api_url: str, chunk_size: int = 50) -> List[str]:
    """
    针对 df 中 status==1 的异常日志进行批量 LLM 分析总结。

    参数:
        df_abnormal: 异常日志子集（必须包含 'log' 列）
        api_keys: 可用大模型密钥列表
        api_url: LLM 接口地址
        chunk_size: 每次分析的日志条数（默认 50）

    返回:
        List[str]：每块分析的结果文本
    """

    logs = df_abnormal['log'].tolist()
    chunks = [logs[i:i + chunk_size] for i in range(0, len(logs), chunk_size)]

    def analyze_chunk(chunk, api_key, idx):
        prompt = f"""你是一位日志分析专家。请阅读以下异常日志条目，判断其共同模式、潜在根因，并总结问题类型。
        你最终的输出结果，必须符合人的阅读习惯和分析思路，撰写的简单易懂，明确标出出错地方。
        每次错误的标出需要引用来源的log。
        日志列表：
        {chr(10).join([f"- {log}" for log in chunk])}
        
        请输出如下格式：
        1. 主要异常模式（如关键字/错误码）
        2. 可能的根因推测
        3. 修复建议（如有）
        （分析尽量简洁明确）"""
        prompt = f"""你是一位专业的日志分析专家，请基于以下日志条目，提取其中的异常模式、潜在根因，并提出清晰的修复建议。
        
        请注意以下要求：
        - **必须**使用人的自然语言习惯进行简洁清晰的表达，避免冗长术语；
        - **每一个异常判断必须引用原始日志内容**，说明异常来源；
        - 分析结果应**按逻辑分段列出**，便于阅读与排查。
        
        请严格按照以下格式输出：
        1. **主要异常模式**（列出关键字、错误码等，并引用相应日志）
        2. **可能的根因推测**（用简洁明确的语言说明可能原因）
        3. **修复建议**（提出可行的排查与修复方向，如有）
        
        以下是异常日志列表：
        {chr(10).join([f"- {log}" for log in chunk])}
        """
        payload = {
            "model": "THUDM/GLM-4-9B-0414",
            "stream": False,
            "max_tokens": 4096,
            "temperature": 0.2,
            "messages": [{"role": "user", "content": prompt}]
        }
        headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

        try:
            response = post_with_retry(payload, headers, api_url)
            result = response['choices'][0]['message']['content']
            return f"【第 {idx + 1} 批异常日志分析】\n{result}\n"
        except Exception as e:
            return f"【第 {idx + 1} 批异常日志分析失败】{str(e)}\n"

    results = []
    with ThreadPoolExecutor(max_workers=min(5, len(api_keys))) as executor:
        futures = [
            executor.submit(analyze_chunk, chunk, api_keys[i % len(api_keys)], i)
            for i, chunk in enumerate(chunks)
        ]
        for fut in tqdm(as_completed(futures), total=len(futures), desc="异常日志分析中"):
            results.append(fut.result())

    return results


In [2]:
df = pd.read_csv("all_log_csv_final_processed.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

In [4]:
df.head()

,source,log,timestamp,source_type,status
0,logcot/log\log\database.log,2025-04-23 14:42:36.060 [INFO] database P00000...,2025-04-23 14:42:36,database.log,1
1,logcot/log\log\database.log,2025-04-23 14:42:36.073 [INFO] database P00000...,2025-04-23 14:42:36,database.log,1
2,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
3,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
4,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1


In [6]:
df[df['timestamp'].dt.date == pd.to_datetime('2025-04-23').date()].head()

,source,log,timestamp,source_type,status
0,logcot/log\log\database.log,2025-04-23 14:42:36.060 [INFO] database P00000...,2025-04-23 14:42:36,database.log,1
1,logcot/log\log\database.log,2025-04-23 14:42:36.073 [INFO] database P00000...,2025-04-23 14:42:36,database.log,1
2,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
3,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
4,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1


In [7]:
types = df['source_type'].unique()
types

array(['database.log', 'log_controller_0_Event.txt'], dtype=object)

In [9]:
# 选出异常日志
abnormal_df = df[(df['status'] == 1) & (df['timestamp'].dt.date == pd.to_datetime('2025-04-23').date())]
abnormal_df = df[(df['status'] == 1) & (df['timestamp'].dt.date == pd.to_datetime('2025-04-23').date()) & (
            df['source_type'] == types[0])]
abnormal_df

,source,log,timestamp,source_type,status
0,logcot/log\log\database.log,2025-04-23 14:42:36.060 [INFO] database P00000...,2025-04-23 14:42:36,database.log,1
1,logcot/log\log\database.log,2025-04-23 14:42:36.073 [INFO] database P00000...,2025-04-23 14:42:36,database.log,1
2,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
3,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
4,logcot/log\log\database.log,2025-04-23 14:42:48.023 [INFO] database P00000...,2025-04-23 14:42:48,database.log,1
...,...,...,...,...,...
406,logcot/log\log\database.log,2025-04-23 15:48:47.035 [INFO] database P00000...,2025-04-23 15:48:47,database.log,1
412,logcot/log\log\database.log,2025-04-23 15:49:58.307 [INFO] database P00000...,2025-04-23 15:49:58,database.log,1
418,logcot/log\log\database.log,2025-04-23 15:51:09.085 [INFO] database P00000...,2025-04-23 15:51:09,database.log,1
424,logcot/log\log\database.log,2025-04-23 15:52:22.119 [INFO] database P00000...,2025-04-23 15:52:22,database.log,1


In [28]:
abnormal_df = df[(df['timestamp'].dt.date == pd.to_datetime('2025-04-23').date()) & (df['source_type'] == types[0])]

API_KEYS = [
    "sk-dpadryupxccpbkigoduasfosszucawczlmfraqhtevaxlokx",
    "sk-kbucebwhrsoimqttlosgtxncyvmuvdyioncbadavayiovrns",
    "sk-zzuykfebwbxkurfftzuvujqpwqzxxljegnxhhwtzddvwiigf",
    "sk-qgsqryixuqdmtzkgubxpvdzollysgtonnvcrwmikwegmaogn",
    "sk-hvxqvahoplbhdadwtaomisdamxqhquvummcfpvlafeovpqus",
]
# 分析这些异常日志
results = analyze_abnormal_logs(
    df_abnormal=abnormal_df,
    api_keys=API_KEYS,
    api_url="https://api.siliconflow.cn/v1/chat/completions"
)

# 打印结果（也可以写入文件）
for r in results:
    print(r)


异常日志分析中: 100%|██████████| 10/10 [00:53<00:00,  5.36s/it]

【第 4 批异常日志分析】

好的，以下是对日志条目的分析：

---

### 1. 主要异常模式
- **关键字**：`socket_err_should_retry errno:104`、`Reached the max session limit`
- **时间点**：集中在 `2025-04-23 15:04:26` 和 `15:08:29` 及之后的多个时间点
- **共同特征**：多个数据库实例（`database P0000010414`）在同一时间段内出现相同的错误

### 2. 可能的根因推测
- **`socket_err_should_retry errno:104`**：通常表示网络套接字错误（如连接超时或网络中断），可能是由于网络不稳定或服务器负载过高导致的连接问题。
- **`Reached the max session limit`**：表示数据库会话数达到上限，可能是由于并发请求过多或会话管理不当导致的。
- **时间关联**：两个错误在时间上高度相关，可能是一个问题的连锁反应。例如，网络问题导致部分连接失败，而服务器资源（如会话数）被耗尽，进一步加剧了问题。

### 3. 修复建议
1. **检查网络连接**：确认服务器和客户端之间的网络稳定性，修复可能的网络中断或延迟问题。
2. **增加会话限制**：如果网络问题无法立即解决，考虑临时增加数据库会话限制，以缓解并发压力。
3. **优化资源分配**：检查服务器资源使用情况，如CPU、内存和磁盘I/O，确保数据库有足够的资源处理请求。
4. **监控和日志记录**：加强监控和日志记录，以便更早发现和响应类似问题。

---

### 总结
这些日志条目揭示了数据库在特定时间段内面临网络连接和会话资源双重压力的问题。建议优先解决网络问题，并考虑临时调整会话限制以缓解压力，同时加强系统监控和资源管理。

【第 1 批异常日志分析】

### 1. 主要异常模式
- **关键字**：`socket_err_should_retry`、`errno:104`、`errno:38`
- **错误码**：
  - `errno:104`：通常表示“连接已被拒绝”（Connection refused），可能由于目标主机不可达或服务未启动。
  - `errno:38`：通常表示“地址错

In [29]:
abnormal_df = df[(df['timestamp'].dt.date == pd.to_datetime('2025-04-23').date()) & (df['source_type'] == types[1])]
API_KEYS = [
    "sk-dpadryupxccpbkigoduasfosszucawczlmfraqhtevaxlokx",
    "sk-kbucebwhrsoimqttlosgtxncyvmuvdyioncbadavayiovrns",
    "sk-zzuykfebwbxkurfftzuvujqpwqzxxljegnxhhwtzddvwiigf",
    "sk-qgsqryixuqdmtzkgubxpvdzollysgtonnvcrwmikwegmaogn",
    "sk-hvxqvahoplbhdadwtaomisdamxqhquvummcfpvlafeovpqus",
    "sk-ewiesnftaeekmhcjixsvnjmgnqldmbafpmjcqcngolcpbyws"
]
# 分析这些异常日志
results = analyze_abnormal_logs(
    df_abnormal=abnormal_df,
    api_keys=API_KEYS,
    api_url="https://api.siliconflow.cn/v1/chat/completions"
)

# 打印结果（也可以写入文件）
for r in results:
    print(r)


异常日志分析中: 100%|██████████| 4/4 [00:17<00:00,  4.34s/it]

【第 4 批异常日志分析】

好的，我们来分析一下这些日志条目：

1.  **主要异常模式**
    *   本日志列表**本身不包含明显的异常模式**。所有日志条目都是标准的“成功”事件（Informational级别），记录了用户登录和登出操作。日志中没有出现错误关键字、错误码或异常状态。

2.  **可能的根因推测**
    *   由于日志条目本身是正常的成功事件，因此**不存在根因**。这些日志只是记录了用户 `mm_user` 从 IP 地址 `127.0.0.1` 在 04:01:01 到 04:07:11 之间进行了两次登录和两次登出的操作，以及用户 `admin` 从 IP 地址 `10.33.36.227` 在 12:19:34 进行了一次登录成功。

3.  **修复建议**
    *   **无需修复**。这些日志条目是正常的，没有指示任何问题或错误。如果目标是分析异常，那么这些日志不适合，需要提供包含错误信息的日志。

【第 1 批异常日志分析】

### 1. 主要异常模式（如关键字/错误码）
- **关键字**：`UltraPath push information from host (...) is not received at the preset time point or fails to be processed.`
- **错误码**：`0x100F0015001C`（多次出现）
- **其他异常**：`FC host port (...) is disconnected.`（一次）

### 2. 可能的根因推测
- **网络连接问题**：日志中多次出现 `UltraPath push information` 未收到的情况，可能由于网络中断或延迟导致主机与存储设备之间的通信失败。
- **设备故障**：`FC host port is disconnected.` 提示了物理连接问题，可能是光纤连接松动或设备故障。
- **配置错误**：如果 `UltraPath` 信息未按时接收，可能是配置了错误的超时时间或参数。

### 3. 修复建议
- **检查物理连接**：确认 FC 光纤连接是否牢固，特别是 `FC host port (controller enclosure CTE0, -- contro